In [57]:
%load_ext autoreload
%autoreload 2
from src.VLLMServer import launch_command,get_client_dict,ask_llm,ask_llm_prompt
from src.code_utils import extract_code_block
from src.process_openmath_ja import parse_record,eval_answer
import yaml

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
conf_path="config.yaml"

with open(conf_path,"r") as f:
    conf=yaml.safe_load(f.read())
client_dict=get_client_dict(conf)

print("vllmサーバーの実行コマンド")
print(launch_command(conf))

#client_dict

vllmサーバーの実行コマンド
export CUDA_VISIBLE_DEVICES=1
python -m vllm.entrypoints.openai.api_server --model microsoft/Phi-3-medium-128k-instruct --max-model-len 4000 --port 8001 --gpu-memory-utilization 0.4 




In [68]:
model_name="microsoft/Phi-3-medium-128k-instruct"
pre_command="""次の問題を解きなさい｡出力事項は以下のとおりである｡
#解説: 問題を解くための基本的な考え方を日本語で出力する｡
#Python: 計算プログラム｡以下の例を参考に､出力形式を守ること｡
```
#a+bの計算を行う
a=1
b=2
print(a+b)
```
#問題
"""

In [65]:
#load dataset
from datasets import load_dataset
ds=load_dataset("kunishou/OpenMathInstruct-1-1.8m-ja",split="train",streaming=False)

In [105]:
ds=ds.shuffle()

In [106]:
len(ds)

1825008

In [103]:

record=ds[0]

#問題文と回答の取得
question,answer=parse_record(record)

#promptの生成
prompt=pre_command+question
res=ask_llm_prompt(client_dict,model_name,prompt)

#生成されたコードを実際にまわして評価する
explanation_block=res[:res.find("```")]
code_block=extract_code_block(res)

#真偽判定
is_correct_answer=eval_answer(code_block,answer,verbose=False)

generated_prompt=explanation_block+"\n"+code_block
if is_correct_answer:
    record["answer"]=generated_prompt

In [104]:
record

{'index': 72867,
 'dataset': 'math',
 'question': 'One piece of string is 1.5 inches long and another piece of string is 4.5 inches long. What is the average length, in inches, of these two pieces of string?',
 'generated_solution': 'The average length is simply the sum of the lengths of the two strings divided by 2.\n<llm-code>\nsum_length = 1.5 + 4.5\navg_length = sum_length / 2\navg_length\n</llm-code>\n<llm-code-output>\n3.0\n</llm-code-output>\nSo the average length is \\\\boxed{3}.',
 'question_ja': '1つの弦の長さは1.5インチで、もう1つの弦の長さは4.5インチです。これらの2つの弦の平均長さ、インチの平均長はどれくらいですか？',
 'generated_solution_ja': '平均長は、2つの文字列の長さを2で割った単純な長さです。<llm-code>\nsum_length = 1.5 + 4.5\navg_length = sum_length / 2\navg_length\n</llm-code><llm-code-output>\n3.0\n</llm-code-output>\nしたがって、平均長は\\\\ boxed {3}です。',
 'answer': '#解説: 2つの弦の長さの平均を求めるには、それらの長さを加算して、2で割る必要があります。この場合、1.5インチと4.5インチの2つの弦があります。\n\n#Python:\n\n#弦の長さ\na = 1.5\nb = 4.5\n\n#平均長計算\naverage_length = (a + b) / 2\n\n#結果の出力\nprint(average_length)'}